In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import QuantLib as ql
from datetime import datetime
import logging

In [3]:
data = pd.read_excel('data/processed/USD/2013-08-26_output.xlsx')
data.head()

,Type,CD,Effective,Maturity,Rate 1,Leg 1,Rate 2,Leg 2,Curr,Not.,...,Trade Time,PF 1,PF 2,RF 1,RF 2,ID,Othr Pmnt,T,Src,Trade Date
0,IRS Fix-Float,TR,09/03/2013,09/01/2018,176.0,FIXED,NaN,USD-LIBOR-BBA,USD,4000000,...,08/26/2013 20:00:00,1M,1M,1M,NaN,4887565,NaN,5Y,DTCC,2013-08-26
1,IRS Fix-Float,TR,09/03/2013,09/01/2023,295.0,FIXED,NaN,USD-LIBOR-BBA,USD,3000000,...,08/26/2013 20:00:00,1M,1M,1M,NaN,4887566,NaN,10Y,DTCC,2013-08-26
2,IRS Fix-Float,TR,09/03/2013,09/01/2023,295.0,FIXED,NaN,USD-LIBOR-BBA,USD,4000000,...,08/26/2013 20:00:00,1M,1M,1M,NaN,4887567,NaN,10Y,DTCC,2013-08-26
3,IRS Fix-Float,TR,09/03/2013,09/01/2023,295.0,FIXED,NaN,USD-LIBOR-BBA,USD,1000000,...,08/26/2013 20:00:00,1M,1M,1M,NaN,4887568,NaN,10Y,DTCC,2013-08-26
4,IRS Fix-Float,TR,09/03/2013,09/01/2023,295.0,FIXED,NaN,USD-LIBOR-BBA,USD,7000000,...,08/26/2013 20:00:00,1M,1M,1M,NaN,4887569,NaN,10Y,DTCC,2013-08-26


In [4]:
curve = pd.read_excel('data/raw/curves/USD/usd_08262013.xlsx')
curve.head()

,Tenor,CUSIP,Description,Yield,Source,Update
0,3M,US0003M Index,US0003M Index,0.26210,CMPN,08/23/13
1,6M,EDZ13 Comdty,EDZ13 Comdty,0.30877,BGN,08/26/13
2,9M,EDH14 Comdty,EDH14 Comdty,0.36707,BGN,08/26/13
3,12M,EDM14 Comdty,EDM14 Comdty,0.44479,BGN,08/26/13
4,15M,EDU14 Comdty,EDU14 Comdty,0.54693,BGN,08/26/13


In [5]:
def build_helpers(yield_curve_df):
    rate_helpers = []
    for _, row in yield_curve_df.iterrows():
        tenor, description, rate, source, update = row['Tenor'], row['Description'], row['Yield'], row['Source'], parse_date(row['Update'])

        if 'US000' in description:
            print(row)
            rate_helpers.append(ql.DepositRateHelper(ql.QuoteHandle(ql.SimpleQuote(rate/100)), 
                                                     ql.Period(tenor), 
                                                     2, 
                                                     ql.UnitedStates(), 
                                                     ql.ModifiedFollowing, 
                                                     False, 
                                                     ql.Actual360()))
        elif 'ED' in description:
            print(row)
            price = 100 - rate  # Convert quote to price
            imm_date = ql.IMM.nextDate(update)
            rate_helpers.append(ql.FuturesRateHelper(price, imm_date, ql.USDLibor(ql.Period(tenor))))
        elif 'USSWAP' in description:
            print(row)
            rate_helpers.append(ql.SwapRateHelper(ql.QuoteHandle(ql.SimpleQuote(rate/100)), 
                                                  ql.Period(tenor), ql.UnitedStates(), 
                                                  ql.Annual, 
                                                  ql.Unadjusted, 
                                                  ql.Thirty360(ql.Thirty360.BondBasis), 
                                                  ql.USDLibor(ql.Period('3M'))))
    
    return rate_helpers

In [6]:
def parse_date(date_string):
    formats = ["%m/%d/%Y", "%m/%d/%y"]

    for date_format in formats:
        try:
            return ql.DateParser.parseFormatted(date_string, date_format)
        except:
            pass

In [9]:
rate_helpers = build_helpers(curve)

Tenor                     3M
CUSIP          US0003M Index
Description    US0003M Index
Yield                 0.2621
Source                  CMPN
Update              08/23/13
Name: 0, dtype: object
Tenor                    6M
CUSIP          EDZ13 Comdty
Description    EDZ13 Comdty
Yield               0.30877
Source                  BGN
Update             08/26/13
Name: 1, dtype: object
Tenor                    9M
CUSIP          EDH14 Comdty
Description    EDH14 Comdty
Yield               0.36707
Source                  BGN
Update             08/26/13
Name: 2, dtype: object
Tenor                   12M
CUSIP          EDM14 Comdty
Description    EDM14 Comdty
Yield               0.44479
Source                  BGN
Update             08/26/13
Name: 3, dtype: object
Tenor                   15M
CUSIP          EDU14 Comdty
Description    EDU14 Comdty
Yield               0.54693
Source                  BGN
Update             08/26/13
Name: 4, dtype: object
Tenor                   18M
CUSIP      

In [7]:
def build_yield_curve(rate_helpers, evaluation_date):
    yield_curve = ql.PiecewiseFlatForward(evaluation_date, rate_helpers, ql.Actual365Fixed())
    yield_curve.enableExtrapolation()
    return yield_curve

In [8]:
def price_swaps(transaction_df, yield_curve, index):
    results = []
    for _, row in transaction_df.iterrows():
        try:
            effective_date, maturity_date, rate_1, leg_1, rate_2, leg_2, currency, notional, payment_frequency_1, payment_frequency_2 = parse_date(row['Effective']), parse_date(row['Maturity']), row['Rate 1'], row['Leg 1'], row['Rate 2'], row['Leg 2'], row['Curr'], row['Not.'], row['PF 1'], row['PF 2']
            if payment_frequency_1 == '1T' or payment_frequency_2 == '1T' or maturity_date <= effective_date:
                continue
            fixed_leg_frequency = ql.Period(payment_frequency_1 if leg_1 == 'FIXED' else payment_frequency_2)
            float_leg_frequency = ql.Period(payment_frequency_1 if leg_1 != 'FIXED' else payment_frequency_2)

            fixed_rate = rate_1 if leg_1 == 'FIXED' else rate_2
            fixed_rate /= 100 if fixed_rate > 10 else 1

            float_rate = rate_1 if leg_1 != 'FIXED' else rate_2
            float_rate /= 100 if float_rate > 10 else 1

            if pd.isnull(float_rate):
                float_rate = 0

            index = index.clone(ql.YieldTermStructureHandle(yield_curve))

            swap_engine = ql.DiscountingSwapEngine(ql.YieldTermStructureHandle(yield_curve))

            fixed_schedule = ql.Schedule(effective_date, 
                                        maturity_date, 
                                        fixed_leg_frequency, 
                                        ql.UnitedStates(), 
                                        ql.ModifiedFollowing, 
                                        ql.ModifiedFollowing, 
                                        ql.DateGeneration.Backward, 
                                        True)
            float_schedule = ql.Schedule(effective_date, 
                                         maturity_date, 
                                        float_leg_frequency, 
                                        ql.UnitedStates(), 
                                        ql.ModifiedFollowing, 
                                        ql.ModifiedFollowing, 
                                        ql.DateGeneration.Backward, 
                                        True)
            swap = ql.VanillaSwap(ql.VanillaSwap.Payer, 
                                notional, 
                                fixed_schedule, 
                                fixed_rate/100, 
                                ql.Thirty360(ql.Thirty360.BondBasis), 
                                float_schedule, 
                                index, 
                                float_rate/100, 
                                index.dayCounter())
            swap.setPricingEngine(swap_engine)
            fair_rate = swap.fairRate()*100
            difference = (fixed_rate - fair_rate)*100

            row['Fair Rate'] = fair_rate
            row['Difference'] = difference
            results.append(row)

        except Exception as e:
            logging.exception(f"Error processing swap:\n{row}\nError: {str(e)}")
            continue
    
    return pd.DataFrame(results)

In [10]:
evaluation_date = '2013-08-26'
evaluation_date_dt = datetime.strptime(evaluation_date, '%Y-%m-%d')
evaluation_date = ql.Date(evaluation_date_dt.day, evaluation_date_dt.month, evaluation_date_dt.year)
uk_calendar = ql.UnitedKingdom()
if not uk_calendar.isBusinessDay(evaluation_date):
    evaluation_date = uk_calendar.adjust(evaluation_date)
ql.Settings.instance().evaluationDate = evaluation_date
yield_curve = build_yield_curve(rate_helpers, evaluation_date)
index = ql.USDLibor(ql.Period('3M'), ql.YieldTermStructureHandle(yield_curve))

In [11]:
historical_fixings_data = pd.read_excel('data/raw/historical_fixings/USD_LIBOR_fixings.xlsx')
# Add historical fixings to the index
for _, row in historical_fixings_data.iterrows():
    fixing_date_dt = row['Date']
    fixing = row['PX_ASK'] / 100  # Convert percentage to decimal

    # fixing_date_dt = datetime.strptime(fixing_date_str, "%m/%d/%y")
    ql_fixing_date = ql.Date(fixing_date_dt.day, fixing_date_dt.month, fixing_date_dt.year)

    if ql_fixing_date <= evaluation_date:
        index.addFixing(ql_fixing_date, fixing)

In [11]:
priced_swaps_df = price_swaps(data, yield_curve, index)
priced_swaps_df.head()

,Type,CD,Effective,Maturity,Rate 1,Leg 1,Rate 2,Leg 2,Curr,Not.,...,PF 2,RF 1,RF 2,ID,Othr Pmnt,T,Src,Trade Date,Fair Rate,Difference
0,IRS Fix-Float,TR,09/03/2013,09/01/2018,176.0,FIXED,NaN,USD-LIBOR-BBA,USD,4000000,...,1M,1M,NaN,4887565,NaN,5Y,DTCC,2013-08-26,1.768039,-0.803893
1,IRS Fix-Float,TR,09/03/2013,09/01/2023,295.0,FIXED,NaN,USD-LIBOR-BBA,USD,3000000,...,1M,1M,NaN,4887566,NaN,10Y,DTCC,2013-08-26,2.935892,1.410807
2,IRS Fix-Float,TR,09/03/2013,09/01/2023,295.0,FIXED,NaN,USD-LIBOR-BBA,USD,4000000,...,1M,1M,NaN,4887567,NaN,10Y,DTCC,2013-08-26,2.935892,1.410807
3,IRS Fix-Float,TR,09/03/2013,09/01/2023,295.0,FIXED,NaN,USD-LIBOR-BBA,USD,1000000,...,1M,1M,NaN,4887568,NaN,10Y,DTCC,2013-08-26,2.935892,1.410807
4,IRS Fix-Float,TR,09/03/2013,09/01/2023,295.0,FIXED,NaN,USD-LIBOR-BBA,USD,7000000,...,1M,1M,NaN,4887569,NaN,10Y,DTCC,2013-08-26,2.935892,1.410807


In [12]:
index = ql.Cdor(ql.Period('3M'), ql.YieldTermStructureHandle(yield_curve))


In [13]:
print(index.fixingCalendar())

Canada calendar
